# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Fracture\15 Male (A View...,0
1,../dataset/wrist_xray/Fracture\94 F (A View).jpg,0
2,../dataset/wrist_xray/Fracture\54 Male (A View...,0
3,../dataset/wrist_xray/Normal\3.jpg,1
4,../dataset/wrist_xray/Fracture\45 F (L View).jpg,0


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel1,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

d:\tutorials\automl\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print('MyNet parameters :' ,count_parameters(mymodel1()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 1.158232
SqueezeNet parameters : 1.234344
ShuffleNet parameters : 1.36636
MnastNet parameters : 2.218224
MobileNet parameters : 3.504296
DenseNet parameters : 7.972584


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# MyNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,\
                            lr=lr,skip_tuning=False,aug=1,dim='2D',pca=False)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.904   0.852     0.872
Class 1       0.818   0.880     0.836
macro         0.862   0.864     0.856
weighted      0.878   0.856     0.858
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.932   0.862     0.896
Class 1       0.824   0.908     0.862
macro         0.880   0.886     0.880
weighted      0.892   0.882     0.884
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.940   0.862     0.900
Class 1       0.828   0.924     0.870
macro         0.886   0.894     0.886
weighted      0.900   0.888     0.890
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.930   0.864     0.894
Class 1       0.826   0.912     0.862
macro         0.880   0.888     0.880
weighted      0.892   0.884     0.886
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.85,0.85,0.85,0.85,0.92
1,0.90,0.92,0.92,0.90,0.92
2,0.69,0.74,0.77,0.77,0.82
3,0.92,0.95,0.95,0.95,0.97
4,0.92,0.95,0.95,0.95,0.97
mean,0.856,0.882,0.888,0.884,0.92


In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.686   0.686     0.680
Class 1       0.562   0.558     0.550
macro         0.622   0.624     0.614
weighted      0.640   0.632     0.626
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.840   0.856     0.842
Class 1       0.788   0.764     0.760
macro         0.812   0.810     0.802
weighted      0.822   0.814     0.808
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.862   0.860     0.858
Class 1       0.804   0.808     0.798
macro         0.834   0.834     0.828
weighted      0.844   0.836     0.834
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.668   0.798     0.718
Class 1       0.630   0.446     0.498
macro         0.648   0.622     0.606
weighted      0.656   0.646     0.622
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.56,0.87,0.90,0.69,0.79
1,0.72,0.85,0.85,0.64,0.79
2,0.69,0.87,0.85,0.74,0.87
3,0.74,0.82,0.87,0.55,0.79
4,0.45,0.66,0.71,0.61,0.74
mean,0.632,0.814,0.836,0.646,0.796


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.602   0.556     0.564
Class 1       0.472   0.514     0.474
macro         0.538   0.534     0.516
weighted      0.554   0.528     0.522
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.622   0.596     0.608
Class 1       0.478   0.508     0.492
macro         0.552   0.552     0.550
weighted      0.568   0.560     0.560
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.606   0.620     0.610
Class 1       0.470   0.458     0.460
macro         0.538   0.540     0.534
weighted      0.554   0.548     0.546
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.668   0.688     0.672
Class 1       0.570   0.532     0.538
macro         0.620   0.612     0.606
weighted      0.634   0.620     0.618
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.72,0.69,0.69,0.69,0.77
1,0.59,0.38,0.38,0.56,0.62
2,0.46,0.51,0.59,0.56,0.67
3,0.45,0.61,0.53,0.66,0.74
4,0.42,0.61,0.55,0.63,0.74
mean,0.528,0.56,0.548,0.62,0.708


In [11]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.648   0.684     0.642
Class 1       0.370   0.420     0.384
macro         0.508   0.552     0.516
weighted      0.540   0.584     0.544
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.568   0.594     0.574
Class 1       0.442   0.402     0.412
macro         0.504   0.500     0.496
weighted      0.522   0.524     0.516
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.588   0.612     0.596
Class 1       0.472   0.432     0.444
macro         0.528   0.524     0.520
weighted      0.542   0.544     0.538
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.584   0.584     0.572
Class 1       0.370   0.374     0.362
macro         0.478   0.482     0.466
weighted      0.504   0.498     0.488
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.64,0.64,0.62,0.62,0.72
1,0.64,0.62,0.56,0.51,0.64
2,0.56,0.59,0.67,0.36,0.72
3,0.53,0.53,0.53,0.61,0.61
4,0.55,0.24,0.34,0.39,0.63
mean,0.584,0.524,0.544,0.498,0.664


# ShuffleNet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.560   0.592     0.572
Class 1       0.418   0.388     0.398
macro         0.488   0.488     0.484
weighted      0.506   0.512     0.504
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.572   0.620     0.588
Class 1       0.464   0.378     0.406
macro         0.516   0.498     0.498
weighted      0.530   0.516     0.516
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.588   0.638     0.604
Class 1       0.482   0.408     0.430
macro         0.536   0.520     0.518
weighted      0.550   0.540     0.536
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.566   0.666     0.604
Class 1       0.398   0.298     0.326
macro         0.484   0.482     0.468
weighted      0.506   0.512     0.492
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.67,0.38,0.46,0.51,0.64
1,0.46,0.69,0.67,0.59,0.69
2,0.56,0.38,0.41,0.49,0.67
3,0.34,0.55,0.55,0.47,0.61
4,0.53,0.58,0.61,0.50,0.68
mean,0.512,0.516,0.54,0.512,0.658


In [13]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.624   0.608     0.602
Class 1       0.464   0.484     0.458
macro         0.546   0.546     0.528
weighted      0.564   0.546     0.538
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.572   0.682     0.622
Class 1       0.412   0.312     0.352
macro         0.490   0.498     0.486
weighted      0.508   0.524     0.506
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.580   0.696     0.628
Class 1       0.424   0.318     0.354
macro         0.504   0.508     0.490
weighted      0.516   0.528     0.510
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.600   0.692     0.642
Class 1       0.466   0.370     0.410
macro         0.536   0.532     0.526
weighted      0.552   0.558     0.548
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.54,0.54,0.56,0.59,0.69
1,0.44,0.49,0.54,0.56,0.67
2,0.59,0.62,0.54,0.54,0.87
3,0.53,0.42,0.37,0.55,0.53
4,0.63,0.55,0.63,0.55,0.68
mean,0.546,0.524,0.528,0.558,0.688


# Mnasnet

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.692   0.664     0.674
Class 1       0.596   0.610     0.594
macro         0.644   0.638     0.634
weighted      0.656   0.638     0.644
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.750   0.736     0.736
Class 1       0.656   0.666     0.644
macro         0.700   0.698     0.688
weighted      0.716   0.702     0.696
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.754   0.764     0.750
Class 1       0.668   0.650     0.646
macro         0.712   0.708     0.698
weighted      0.728   0.710     0.708
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.724   0.764     0.736
Class 1       0.644   0.592     0.604
macro         0.684   0.678     0.670
weighted      0.698   0.692     0.684
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.51,0.62,0.59,0.59,0.67
1,0.54,0.64,0.72,0.67,0.72
2,0.56,0.67,0.69,0.62,0.74
3,0.74,0.76,0.76,0.76,0.76
4,0.84,0.82,0.79,0.82,0.84
mean,0.638,0.702,0.71,0.692,0.746


In [15]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.568   0.552     0.552
Class 1       0.408   0.426     0.408
macro         0.488   0.490     0.484
weighted      0.504   0.490     0.492
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.584   0.624     0.594
Class 1       0.458   0.398     0.404
macro         0.520   0.514     0.498
weighted      0.536   0.524     0.514
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.554   0.580     0.562
Class 1       0.390   0.368     0.374
macro         0.472   0.474     0.468
weighted      0.490   0.488     0.484
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.536   0.640     0.578
Class 1       0.388   0.280     0.316
macro         0.462   0.462     0.450
weighted      0.480   0.486     0.472
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.59,0.41,0.44,0.28,0.67
1,0.46,0.38,0.46,0.51,0.62
2,0.51,0.64,0.51,0.59,0.67
3,0.47,0.58,0.50,0.50,0.63
4,0.42,0.61,0.53,0.55,0.58
mean,0.49,0.524,0.488,0.486,0.634


# MobileNet

In [16]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.652   0.592     0.588
Class 1       0.496   0.514     0.468
macro         0.574   0.554     0.528
weighted      0.596   0.550     0.536
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.656   0.710     0.678
Class 1       0.526   0.472     0.494
macro         0.590   0.592     0.586
weighted      0.610   0.618     0.610
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.652   0.738     0.692
Class 1       0.540   0.448     0.484
macro         0.596   0.592     0.586
weighted      0.614   0.624     0.608
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.714   0.702     0.694
Class 1       0.570   0.588     0.564
macro         0.642   0.642     0.630
weighted      0.658   0.646     0.640
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.54,0.64,0.54,0.54,0.69
1,0.46,0.62,0.67,0.69,0.74
2,0.56,0.54,0.62,0.74,0.67
3,0.61,0.61,0.66,0.55,0.71
4,0.58,0.68,0.63,0.71,0.79
mean,0.55,0.618,0.624,0.646,0.72


In [17]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.598   0.632     0.612
Class 1       0.470   0.438     0.444
macro         0.532   0.534     0.526
weighted      0.550   0.546     0.540
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.566   0.672     0.608
Class 1       0.444   0.322     0.360
macro         0.506   0.496     0.484
weighted      0.520   0.518     0.504
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.590   0.726     0.650
Class 1       0.450   0.320     0.372
macro         0.522   0.524     0.512
weighted      0.538   0.554     0.534
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.576   0.686     0.616
Class 1       0.408   0.318     0.342
macro         0.490   0.504     0.480
weighted      0.506   0.524     0.500
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.44,0.46,0.46,0.46,0.67
1,0.51,0.59,0.54,0.46,0.69
2,0.67,0.49,0.64,0.64,0.72
3,0.53,0.42,0.47,0.45,0.55
4,0.58,0.63,0.66,0.61,0.74
mean,0.546,0.518,0.554,0.524,0.674


# DenseNet121

In [18]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.668   0.682     0.668
Class 1       0.546   0.538     0.536
macro         0.610   0.610     0.602
weighted      0.624   0.616     0.614
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.752   0.770     0.758
Class 1       0.684   0.660     0.664
macro         0.718   0.714     0.710
weighted      0.728   0.720     0.720
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.752   0.804     0.772
Class 1       0.704   0.636     0.662
macro         0.728   0.718     0.718
weighted      0.738   0.730     0.728
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.754   0.680     0.712
Class 1       0.640   0.714     0.668
macro         0.696   0.698     0.690
weighted      0.714   0.696     0.696
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.82,0.85,0.90,0.85,0.90
1,0.67,0.77,0.74,0.74,0.77
2,0.51,0.59,0.69,0.62,0.74
3,0.61,0.63,0.71,0.45,0.74
4,0.47,0.76,0.61,0.82,0.82
mean,0.616,0.72,0.73,0.696,0.794


In [19]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=lr,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.616   0.718     0.662
Class 1       0.508   0.390     0.442
macro         0.562   0.552     0.552
weighted      0.580   0.582     0.572
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.726   0.730     0.724
Class 1       0.630   0.620     0.622
macro         0.676   0.678     0.672
weighted      0.692   0.680     0.682
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.712   0.726     0.714
Class 1       0.642   0.622     0.622
macro         0.678   0.672     0.666
weighted      0.688   0.674     0.672
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.646   0.700     0.664
Class 1       0.554   0.482     0.490
macro         0.604   0.592     0.576
weighted      0.616   0.596     0.584
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.69,0.59,0.72,0.62,0.64
1,0.62,0.69,0.51,0.46,0.72
2,0.49,0.59,0.64,0.64,0.72
3,0.61,0.71,0.66,0.55,0.68
4,0.50,0.82,0.84,0.71,0.79
mean,0.582,0.68,0.674,0.596,0.71
